In [1]:
import torch
import numpy as np
import torch.optim as optim

ModuleNotFoundError: No module named 'torch'

In [154]:
data = np.genfromtxt('movieReplicationSet.csv', delimiter = ',',skip_header = 1)
p_value = []
count = 0
sum = 0
a = np.empty((1097,477))
for i in range(477):
    temp = data[:,i:i+1]
    temp2  = np.isfinite(data[:,i:i+1])
    temp = temp[temp2]
    avg = np.median(temp)
    #print(avg)
    for j in range(len(temp2)):
        if temp2[j] == True:
            a[j][i] = data[j,i]
        else:
            a[j][i] = avg
a = np.abs(a.T) 
a.shape
data=a.T
print(data.shape)

(1097, 477)


In [201]:
from torch import nn

class jani(torch.nn.Module):

  def __init__(self):
    super().__init__()
    # TODO

    self.fc1 = nn.Linear(77,200)

    self.fc2 = nn.Linear(200,400)

    self.fc3 = nn.Linear(400,600)

    self.fc4=nn.Linear(600,400)
    
    self.dropout = nn.Dropout(0.2)
    self.relu = nn.ReLU()
    self.sig=nn.Sigmoid()

  def forward(self, x):

    # TODO
    out=self.relu(self.fc1(x))
    out=self.dropout(out)
    out=self.fc2(out)
    out=self.relu(out)
    # out=self.dropout(out)
    out=self.relu(self.fc3(out))
    out=self.dropout(out)
    out=self.relu(self.fc4(out))
    return out
    pass


In [202]:
train_data, test_data = torch.utils.data.random_split(data, [1000, 97])
train_data_loader=torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True, num_workers=4)
test_data_loader=torch.utils.data.DataLoader(test_data, batch_size=1, shuffle=True, num_workers=4)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [203]:
#training
model=jani()
criterion = nn.MSELoss()
# criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = model.parameters(),lr=0.0001)

for val in train_data:
  input=torch.tensor(val[400:]).float()
  labels=torch.tensor(val[:400]).float()
  labels=torch.round(labels)
  pred=model(input)
  # pred=torch.round(pred)
  pred=pred.unsqueeze(0)
  lables=labels.unsqueeze(0).unsqueeze(0)
  # pred=torch.round(pred.unsqueeze(0))
  # print('pred labels',pred[:10])
  # print('true labels',labels[:10])
  # print('pred labels',pred)
  # print('true labels',labels)
  optimizer.zero_grad()
  # for i in range(len(pred)):
  #   print(pred[0])
  #   loss+=criterion(pred[i].unsqueeze(0),labels[i].unsqueeze(0))
  loss=criterion(pred,labels)
  loss.backward()
  optimizer.step()

  # print(loss)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:520: UserWarning: Using a target size (torch.Size([400])) that is different to the input size (torch.Size([1, 400])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [204]:
#testing
count=0
acc=[]
for val in test_data:
  count=0
  input=torch.tensor(val[400:]).float()
  labels=torch.tensor(val[:400]).float().unsqueeze(0)
  pred=model(input)
  pred=torch.round(pred)
  labels=torch.round(labels)
  diff=pred-labels
  count=400-torch.count_nonzero(diff)
  # acc=count/400
  acc.append(count/400)
  # print(count/400)
acc=torch.tensor(acc)
print('accuracy is:-',torch.mean(acc))




accuracy is:- tensor(0.6686)
